# Capstone Project - The battle of the flavours (Week 5) Complete Report
### Applied Data Science Capstone by IBM/Coursera
### Problem description business case.

## Table of contents
* **[1. Introduction: Business Case Problem](#introduction)**
* **[2. Data](#data)**
* **[3. Methodology](#Methodology)**
* **[4. Analysis](#Analysis)**
* **[5. Results and Discussion](#Results)**
* **[6. Conclusions and way forward](#Conclusions)**



## 1. Introduction: Business Case <a name="introduction"></a>

In this project we want to idenfify the best location to craft-beer shop around Madrid downtown. Recently many people turned back their taste preferences to these products looking for healthier food and authentic flavours which have been put aside by the industrialized food companies.

This kind of shops have the favour of the public, specially those people with ages around between the thirties to fourties.

The shop will be something different as it targets to sell a highly valuable product for the Spanish population.

We will use data got from the different Madrid downtown districts **to find the most suitable location for the shop**, based on the similar shops around a distance of 500m, the ages distribution of the living population in those districts, **targeting people between 25 to 50 years old**.

Also **the price per square meter** from the different above selected districts, looking for the cheapest one to open the shop as the final criteria to select a place.

## 2. Data <a name="Data"></a>

Based on definition of our problem, factors that will influence our decision are:

* Number of existing craft-beer shops in the neighborhood / district.
* Distance to  other craft-beer shops in the neighborhood or district, if any.
* Age distribution in the districts
* Price in euros per square meter of the houses or shops in the districts, real-state.

Following data sources will be needed to extract/generate the required information:
* Centers of districts will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Geocoder Python Library** 

https://geocoder.readthedocs.io/index.html

* Number of craft-beer shops or similar venues in every neighborhood will be obtained using **Foursquare API**
* Coordinate of Madrid center will be obtained using **Geocoder Python Library** of well known Madrid location (Puerta del Sol).
* Housing prices and demographic information from Madrid districts will be got from Madrid´s city council web page:

https://www.madrid.es/portales/munimadrid/es/Inicio/El-Ayuntamiento/Estadistica/Areas-de-informacion-estadistica/Areas-de-informacion-estadistica/?vgnextfmt=default&vgnextoid=9023c9fa0b23a210VgnVCM2000000c205a0aRCRD&vgnextchannel=b65ef78526674210VgnVCM1000000b205a0aRCRD

* From the different excel files got from the city council web page, the information needed from the districts will be grouped and filtered for later use along with the geographical data and venues data in order to complete the business case analysis.


### Import Libraries needed for the Project

In [1]:
# Import all needed libraries
import os # System files movements
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib as plt
import matplotlib.pyplot as plt

# Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed and imported!')
!pip install geocoder
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!pip install geojson
print('Geopy, Geocoder, and geojson installed and imported!')

# Dependency needed to install file 
!pip install xlrd

import requests # library to handle requests
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import geojson

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    libcblas-3.8.0             |      11_openblas          10 KB  conda-forge


### Load Data from Madrid's City Council
Relevant statistical information has been downloaded from Madrid's City Council, and arranged into one Excel file, which is going to be loaded as a pandas dataframe.

In [125]:
dfMadrid = pd.read_excel('Datos_Madrid_Capstone.xls',sheet_name='Data')
print('Data downloaded and read into a dataframe!')
dfMadrid.head()

Data downloaded and read into a dataframe!


,Distrito,Índice de juventud,ObraNueva (Eur/m2),ObraUsada (Eur/m2)
0,Ciudad de Madrid,70.179326,2624.77,3387.14
1,Centro,51.610072,4521.91,4817.37
2,Arganzuela,65.438845,3670.25,3698.09
3,Retiro,48.715364,3989.25,4457.96
4,Salamanca,47.506659,5674.82,5398.93


### District Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a folium map covering our area of interest a 5km circle centered around Madrid city center (Puerta del Sol).

Let's first find the latitude & longitude of Madrid city center, using specific, well known address and Geocoder

### Read geojson districts geographical information downloaded from Github.

In [134]:
madrid_districts_geo = r'madrid-districts.geojson'  # Para leer el fichero geojson del directorio local.
print('GEOJSON file readed!')

GEOJSON file readed!


In [135]:
data_districts = pd.read_json('/resources/labs/DP0701EN/Capstone/madrid-districts.geojson')
#features = data_districts['features'][0]
data_districts['features'][0]

{'type': 'Feature',
 'properties': {'name': 'Centro',
  'cartodb_id': 1,
  'created_at': '2013-12-02T07:20:26+0100',
  'updated_at': '2013-12-02T07:20:26+0100'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-3.691853, 40.408527],
     [-3.691893, 40.408377],
     [-3.691919, 40.408167],
     [-3.692368, 40.408309],
     [-3.692541, 40.408438],
     [-3.6925939999999997, 40.408467],
     [-3.692657, 40.408475],
     [-3.692796, 40.408454],
     [-3.69398, 40.408134],
     [-3.694102, 40.408179],
     [-3.696566, 40.407529],
     [-3.696634, 40.407509],
     [-3.698068, 40.40711],
     [-3.698772, 40.406915],
     [-3.699676, 40.406683],
     [-3.701244, 40.40625],
     [-3.701663, 40.406245],
     [-3.702577, 40.406379],
     [-3.705198, 40.40686],
     [-3.707094, 40.40725],
     [-3.708229, 40.407459],
     [-3.708605, 40.407511],
     [-3.709974, 40.407742999999996],
     [-3.710369, 40.407846],
     [-3.7107390000000002, 40.407926],
     [-3.712384, 40.408625],
     [-

In [5]:
#data_districts['features'][0]['geometry']['coordinates']
#lista_Coords_Centro = data_districts['features'][0]['geometry']['coordinates']
#lista_Coords_Centro[0][0][0] # Access to coordinates list from each point that defines the polygon around any district.

In [136]:
lista_Coords_Centro = list(geojson.utils.coords(data_districts)) 
len(lista_Coords_Centro[:])
lista_Coords_Centro

[(-3.691853, 40.408527),
 (-3.691893, 40.408377),
 (-3.691919, 40.408167),
 (-3.692368, 40.408309),
 (-3.692541, 40.408438),
 (-3.6925939999999997, 40.408467),
 (-3.692657, 40.408475),
 (-3.692796, 40.408454),
 (-3.69398, 40.408134),
 (-3.694102, 40.408179),
 (-3.696566, 40.407529),
 (-3.696634, 40.407509),
 (-3.698068, 40.40711),
 (-3.698772, 40.406915),
 (-3.699676, 40.406683),
 (-3.701244, 40.40625),
 (-3.701663, 40.406245),
 (-3.702577, 40.406379),
 (-3.705198, 40.40686),
 (-3.707094, 40.40725),
 (-3.708229, 40.407459),
 (-3.708605, 40.407511),
 (-3.709974, 40.407742999999996),
 (-3.710369, 40.407846),
 (-3.7107390000000002, 40.407926),
 (-3.712384, 40.408625),
 (-3.713922, 40.409264),
 (-3.715461, 40.409879),
 (-3.717029, 40.412294),
 (-3.716542, 40.413259),
 (-3.71563, 40.415097),
 (-3.71723, 40.415095),
 (-3.718123, 40.415116),
 (-3.718328, 40.415121),
 (-3.719186, 40.415134),
 (-3.721644, 40.415171),
 (-3.721541, 40.416513),
 (-3.72107, 40.418356),
 (-3.720851, 40.419217),
 (-3

In [ ]:
# Let´s find each district geographical center using coords from the Multipolygons which define each one

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on breweries in each neighborhood.
Foursquare code for brewery or "craft-beer" (in Spanish "Tienda de Cerveza").

In [4]:
!pip install foursquare

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/53/6c/d9/0810f42ef7521037af97032caab9411144ab0efab2aed8300f
Successfully built foursquare


### Define Foursquare Credentials and Version

In [26]:
CLIENT_ID = 'FLZED3BEIVB5HLS3ZMJTNWBSFL3WL0QM4T1SVXW0YI3UU5A4' # your Foursquare ID
CLIENT_SECRET = 'W231AZOJUDH3JZGC4XKVZX215KE33DINLTCVSWZT2D3F11KN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FLZED3BEIVB5HLS3ZMJTNWBSFL3WL0QM4T1SVXW0YI3UU5A4
CLIENT_SECRET:W231AZOJUDH3JZGC4XKVZX215KE33DINLTCVSWZT2D3F11KN


In [6]:
address = 'Puerta del Sol, Madrid'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.4168654 -3.7043021694878


### Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Search for Breweries or craft-beer shops **("Cervecerías in Spanish")**, some Foursquare codes for similar places

Cervecera
50327c8591d4c4b30a586d5d

Tienda de Cerveza
5370f356bcbc57f1066c94c2

We will perform three searches one per category, to get three dataframes which will be merged and cleaned later.

In [7]:
# 1. Search for 
search_query = 'craft-beer'
#search_query = 'beer'
categoryID = ['50327c8591d4c4b30a586d5d','5370f356bcbc57f1066c94c2']
radius = 5000 # 5km radius around center
print(search_query + ' .... OK!')

craft-beer .... OK!


In [27]:
# Loop through the categories ID to get three dataframes
url_0 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryID={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT,categoryID[0])
results_0 = requests.get(url_0).json() # Send the GET Request and examine the results
# Get relevant part of JSON and transform it into a *pandas* dataframe 
# assign relevant part of JSON to venues
cervecera = results_0['response']['venues']
# tranform venues into a dataframe
df_cervecera = json_normalize(cervecera)

url_1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryID={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT,categoryID[1])
results_1 = requests.get(url_1).json() # Send the GET Request and examine the results
# Get relevant part of JSON and transform it into a *pandas* dataframe 
# assign relevant part of JSON to venues
tienda_de_cerveza = results_0['response']['venues']
# tranform venues into a dataframe
df_tienda = json_normalize(tienda_de_cerveza)  

In [28]:
# Merge all together
dframes = [df_cervecera,df_tienda]
df_craftb = pd.concat(dframes, axis = 0)
df_craftb.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,5bf1a840ad910e002c4a046b,La Tienda De La Cerveza - Craft Beer Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1576911699,False,"Calle de las Maldonadas, 5",40.410942,-3.708029,"[{'label': 'display', 'lat': 40.410942, 'lng':...",731,28005,ES,Madrid,Madrid,España,"[Calle de las Maldonadas, 5, 28005 Madrid Madr...",NaN,NaN,NaN
1,5b087bb6b9a5a8002c3477cb,Sam Hop Craft Beer,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",v-1576911699,False,Calle De Lavapies,40.411468,-3.702996,"[{'label': 'display', 'lat': 40.411468, 'lng':...",610,28012,ES,Madrid,Madrid,España,"[Calle De Lavapies, 28012 Madrid Madrid, España]",NaN,NaN,NaN
2,5905bec2a4236221271945b2,Craft 19 - Craft Beer & Pastrami,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1576911699,False,Calle de Vallehermoso 36,40.434100,-3.708280,"[{'label': 'display', 'lat': 40.4341, 'lng': -...",1947,28015,ES,Madrid,Madrid,España,[Calle de Vallehermoso 36 (c/ Fernando el Cató...,c/ Fernando el Católico,418985411,NaN
3,579905c6498e213c8e2b4f62,Monasterio Craft Beer,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1576911699,False,Calle Rodriguez Sanpedro,40.431570,-3.708656,"[{'label': 'display', 'lat': 40.43157, 'lng': ...",1677,28015,ES,Madrid,Madrid,España,"[Calle Rodriguez Sanpedro, 28015 Madrid Madrid...",NaN,NaN,NaN
4,5d87fabc3955b60008e4dc5b,Mayrit craft beer,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",v-1576911699,False,Quero 33,40.391490,-3.760105,"[{'label': 'display', 'lat': 40.39148961918852...",5509,28024,ES,Madrid,Madrid,España,"[Quero 33 (maqueda), 28024 Madrid Madrid, España]",maqueda,NaN,NaN


In [16]:
df_craftb.shape

(80, 19)

In [18]:
df_craftb.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.lat', 'location.lng', 'location.labeledLatLngs',
       'location.distance', 'location.postalCode', 'location.cc',
       'location.city', 'location.state', 'location.country',
       'location.formattedAddress', 'location.crossStreet', 'venuePage.id',
       'location.neighborhood'],
      dtype='object')

In [174]:
# We drop all non-necessary columns
df_craftb_drop = df_craftb.drop(columns=['id', 'referralId', 'hasPerk',
       'location.labeledLatLngs',
       'location.postalCode', 'location.cc',
       'location.city', 'location.state', 'location.country',
       'location.formattedAddress', 'location.crossStreet', 'venuePage.id',
       'location.neighborhood'],axis =1)
#df_craftb_drop = df_craftb.drop(['hasPerk'],axis=1)
df_craftb_drop.head()

,name,categories,location.address,location.lat,location.lng,location.distance
0,La Tienda De La Cerveza - Craft Beer Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...","Calle de las Maldonadas, 5",40.410942,-3.708029,731
1,Sam Hop Craft Beer,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",Calle De Lavapies,40.411468,-3.702996,610
2,Craft 19 - Craft Beer & Pastrami,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",Calle de Vallehermoso 36,40.434100,-3.708280,1947
3,Monasterio Craft Beer,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",Calle Rodriguez Sanpedro,40.431570,-3.708656,1677
4,Mayrit craft beer,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",Quero 33,40.391490,-3.760105,5509


In [175]:
# Remove duplicated entries from dataset
df_craftb_drop.drop_duplicates(subset = ['name'],inplace = True)

In [176]:
df_craft_new = df_craftb_drop.reset_index(drop=True)
df_craft_new.head()

,name,categories,location.address,location.lat,location.lng,location.distance
0,La Tienda De La Cerveza - Craft Beer Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...","Calle de las Maldonadas, 5",40.410942,-3.708029,731
1,Sam Hop Craft Beer,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",Calle De Lavapies,40.411468,-3.702996,610
2,Craft 19 - Craft Beer & Pastrami,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",Calle de Vallehermoso 36,40.434100,-3.708280,1947
3,Monasterio Craft Beer,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",Calle Rodriguez Sanpedro,40.431570,-3.708656,1677
4,Mayrit craft beer,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",Quero 33,40.391490,-3.760105,5509


In [177]:
# Save dataframe: 
df_craft_new.to_csv('craft_beer_Madrid.csv')

#### Let's visualize the craft-beer shops nearby

#### Define information of interest and filter dataframe

In [178]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_craft_new.columns if col.startswith('location.')] # Condition definition
df_filtered = df_craft_new.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

df_filtered

,name,categories,address,lat,lng,distance
0,La Tienda De La Cerveza - Craft Beer Bar,Bar,"Calle de las Maldonadas, 5",40.410942,-3.708029,731
1,Sam Hop Craft Beer,Beer Bar,Calle De Lavapies,40.411468,-3.702996,610
2,Craft 19 - Craft Beer & Pastrami,Sandwich Place,Calle de Vallehermoso 36,40.434100,-3.708280,1947
3,Monasterio Craft Beer,Brewery,Calle Rodriguez Sanpedro,40.431570,-3.708656,1677
4,Mayrit craft beer,Beer Bar,Quero 33,40.391490,-3.760105,5509
5,Bee Beer,Beer Store,"C. Augusto Figueroa, 30",40.422077,-3.696958,850
6,Craft Against The Machine,Beer Bar,"Calle de Embajadores, 31",40.408650,-3.705004,916
7,Madrid Beer Pong,Bar,NaN,40.418987,-3.703614,243
8,Craft Crew Madrid,Beer Store,"Barceló, 6",40.426647,-3.698970,1178
9,Belgian Beer Cafe,Bar,NaN,40.414737,-3.708190,405


In [181]:
# keep only places that include venue category as craft beer of brewery, and anything that is associated with location
breweries = df_filtered.loc[df_filtered['categories'] == 'Brewery',['name','categories','address','lat','lng','distance']]
beer_store = df_filtered.loc[df_filtered['categories'] == 'Beer Store',['name','categories','address','lat','lng','distance']]

In [182]:
# Assembly new dataframe
dummydf= [breweries,beer_store]
craft_beer = pd.concat(dummydf,axis =0).reset_index(drop=True)
craft_beer.head(10)

,name,categories,address,lat,lng,distance
0,Monasterio Craft Beer,Brewery,Calle Rodriguez Sanpedro,40.431570,-3.708656,1677
1,Madrid Shooter Beer,Brewery,Calle Palafox 1,40.430534,-3.700782,1550
2,Beershooter Malasaña,Brewery,"Calle La Palma, 69",40.426713,-3.708091,1142
3,Cervezas Eterna,Brewery,Calle de Eraso 14,40.434117,-3.671035,3411
4,house of beer,Brewery,NaN,40.456413,-3.691424,4535
5,Bee Beer,Beer Store,"C. Augusto Figueroa, 30",40.422077,-3.696958,850
6,Craft Crew Madrid,Beer Store,"Barceló, 6",40.426647,-3.698970,1178
7,The Beer Garden Store,Beer Store,"C. Cardenal Cisneros, 10",40.430335,-3.701842,1513


#### Find the districts to which every shop belongs to. With Geocoder inverse

### Load the data from Madrid's City Council with Real State prices and Population Young Index.

In [184]:
dfMadrid = pd.read_excel('Datos_Madrid_Capstone.xls',sheet_name='Data')

print('Data downloaded and read into a dataframe!')
dfMadrid.head(20)

Data downloaded and read into a dataframe!


,Distrito,Índice de juventud,ObraNueva (Eur/m2),ObraUsada (Eur/m2),lon,lat
0,Ciudad de Madrid,70.179326,2624.77,3387.14,-3.691944,40.418889
1,Centro,51.610072,4521.91,4817.37,-3.707371,40.415347
2,Arganzuela,65.438845,3670.25,3698.09,-3.695403,40.402733
3,Retiro,48.715364,3989.25,4457.96,-3.676729,40.408072
4,Salamanca,47.506659,5674.82,5398.93,-3.677778,40.430000
5,Chamartín,59.347391,4270.04,4776.80,-3.677500,40.453333
6,Tetuán,60.636834,3173.14,3339.84,-3.700000,40.460556
7,Chamberí,44.947219,5912.31,5170.16,-3.697186,40.432792
8,Fuencarral-El Pardo,86.377316,3313.84,3310.43,-3.709722,40.478611
9,Moncloa-Aravaca,66.284360,3916.07,3686.56,-3.718765,40.435151


In [185]:
dfMadrid_sorted = dfMadrid.sort_values(by=['Índice de juventud','ObraNueva (Eur/m2)'],ascending = False)
dfMadrid_sorted.head(10)

,Distrito,Índice de juventud,ObraNueva (Eur/m2),ObraUsada (Eur/m2),lon,lat
18,Villa de Vallecas,154.647029,2153.66,2133.84,-3.621350,40.379600
19,Vicálvaro,129.278588,1517.02,2032.67,-3.608060,40.404200
21,Barajas,103.172414,3517.48,3160.19,-3.584890,40.470196
16,Hortaleza,95.558854,3016.09,3539.82,-3.640482,40.469457
17,Villaverde,95.329988,1546.41,1424.43,-3.709356,40.345925
12,Usera,91.497041,1661.17,1789.41,-3.706856,40.381336
20,San Blas-Canillejas,88.419045,3022.19,2238.81,-3.612764,40.426001
8,Fuencarral-El Pardo,86.377316,3313.84,3310.43,-3.709722,40.478611
13,Puente de Vallecas,78.008536,1617.61,1644.77,-3.669059,40.398204
11,Carabanchel,76.720403,2150.63,1944.43,-3.727989,40.383669


#### Create two plots, one overlapping the craft-beer locations to the Young Index and another one overlapping the real state prices with the craft-beer locations

In [189]:
craftbeer_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Puerta del Sol.

# add a red circle marker to represent the Puerta del Sol (km 0)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Puerta del Sol',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(craftbeer_map)

# add the Breweries as blue circle markers
for lat, lng, label in zip(craft_beer.lat, craft_beer.lng, craft_beer.categories):
    folium.features.CircleMarker( 
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(craftbeer_map)

# Generate Markers for the centers of the districts 
for lat, lon, label in zip(dfMadrid.lat, dfMadrid.lon, dfMadrid.Distrito):
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.8
    ).add_to(craftbeer_map)

# generate choropleth map using the Indice de Juventud "Young Index"
craftbeer_map.choropleth(
    geo_data=madrid_districts_geo,
    data=dfMadrid,
    columns=['Distrito', 'Índice de juventud'],
    key_on='feature.properties.name',
    fill_color='YlGn', 
    fill_opacity= 0.5, 
    line_opacity= 0.2,
    legend_name = 'Young Index'
)

# display map
craftbeer_map

In [191]:
craftbeer_map2 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Puerta del Sol.

# add a red circle marker to represent the Puerta del Sol (km 0)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Puerta del Sol',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(craftbeer_map2)

# add the Breweries as blue circle markers
for lat, lng, label in zip(craft_beer.lat, craft_beer.lng, craft_beer.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(craftbeer_map2)
    
# Generate Markers for the centers of the districts 
for lat, lon, label in zip(dfMadrid.lat, dfMadrid.lon, dfMadrid.Distrito):
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.8
    ).add_to(craftbeer_map2)
    
# generate choropleth map using the "Precio de Obra Nueva" real-state prices
craftbeer_map2.choropleth(
    geo_data=madrid_districts_geo,
    data=dfMadrid,
    columns=['Distrito', 'ObraNueva (Eur/m2)'],
    key_on='feature.properties.name',
    fill_color='BuPu', 
    fill_opacity= 0.3, 
    line_opacity= 0.2,
    legend_name = 'Real State prices (Eur/m2)'
)

# display map
craftbeer_map2

### First Conclusions from data wrangling 

At first glance it looks like that the optimum places for our investors in which we can combine a high Young Index and moderate real state prices are in the border line between **'Centro' and Moncloa'** districts. *Always based on our initial condition of finding this place in a radius of 5km from Madrid City Center*.

## 3. Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Madrid downtown with low number of Craft Beer shops. We will limit our analysis to area ~ 5km around city center.

In first step we have collected the required **data: location and type (category) of every Beer related place within 5km from Madrid center** ("Puerta del Sol). We have also **identified bars or other similar places where beer is sold** (according to Foursquare categorization). But these were removed keeping only breweries and craft-beer shops.

Also we paired this information with Madrid´s City Council information about age distribution and real-state prices.

In the methodology section we will look for the most promising addresses in a 5km circle with a reasonable real-state prices and highest Young Index.

Second step in our analysis will be calculation and exploration of '**craft beer density**' across different areas of Madrid - we will use **heatmaps** to identify a few promising areas close to center with low number of shops in general focus our attention on those areas. Also we will paid attention to the Young Index of the living population on those areas and real-state places to identify the best suitable place to open such a shop

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no craft beef shops in radius of 500 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## 4. Analysis <a name="analysis"></a>

### 4.1 k-means initialization <a name="k-means"></a>
#### We have 2 clusters one for breweries and another for beer shops

In [ ]:
k_means = KMeans(init="k-means++", n_clusters=2, n_init=12)

In [ ]:
k_means.fit(data)

In [ ]:
k_means_labels = k_means.labels_
k_means_labels

In [ ]:
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

## 5. Results and Discussion <a name="results"></a>

## 6. Conclusions and way forward <a name="conclusion"></a>